In [ ]:
import numpy as np
import librosa
import os
from sklearn.mixture import GaussianMixture

In [ ]:
def preprocess(utterance):
    max_db = np.max(librosa.amplitude_to_db(utterance))
    intervals = librosa.effects.split(utterance, top_db=41, ref=max_db)

    new_utterance = np.empty((0,))
    for i in intervals:
        new_utterance = np.hstack((new_utterance, utterance[i[0]:i[1]]))

    return new_utterance

def get_features(utterance: np.ndarray, sr: int, n_mfccs: int=20):

    mfccs  = librosa.feature.mfcc(utterance, sr, n_mfcc=n_mfccs)
    mfccs1 = librosa.feature.delta(mfccs)
    mfccs2 = librosa.feature.delta(mfccs, order=2)

    features = np.hstack((mfccs.T, mfccs1.T, mfccs2.T))
    return features

In [ ]:
def get_gmm(directory: str) -> GaussianMixture:

    features = np.empty((0, 60))
    for filepath in os.listdir(directory):
        utt, sr = librosa.load(os.path.join(directory, filepath))

        utt = preprocess(utt)
        if utt.shape[0] == 0:
            continue

        utt = librosa.util.normalize(utt)
        features = np.vstack((features, get_features(utt, sr)))

    gmm = GaussianMixture(n_components=128,
                          covariance_type='diag',
                          n_init=3,
                          max_iter=200)

    gmm.fit(features)
    return gmm

def test_gmm(gmms:list[GaussianMixture],
             labels: list[str],
             correct_label: str,
             directory: str):

    results = []
    for filepath in os.listdir(directory):
        utt, sr = librosa.load(os.path.join(directory, filepath))

        new_utt = preprocess(utt)
        if new_utt.shape[0] < 7000:
            new_utt = utt

        new_utt = librosa.util.normalize(new_utt)
        # print(f'{filepath = }, {utt.shape = }')
        features = get_features(new_utt, sr)

        prediction = []
        for gmm, label in zip(gmms, labels):
            prediction.append(gmm.score(features))

        results.append(labels[np.argmax(prediction)] == correct_label)

    results = np.array(results, dtype=int)

    return np.sum(results) / results.shape[0]

In [ ]:
print('Creating GMMs...')
gmm_female = get_gmm('./dataset/train_data/female')
print('Female GMM done.')
gmm_male   = get_gmm('./dataset/train_data/male')
print('Make GMM done.')

print('Testing...')
result_female = test_gmm([gmm_female, gmm_male],
                         ['f', 'm'],
                         'f',
                         './dataset/test_data/female')

print(f'Female accuracy: {result_female}')

result_male = test_gmm([gmm_female, gmm_male],
                       ['f', 'm'],
                       'm',
                       './dataset/test_data/male')

print(f'Male accuracy: {result_male}')